In [ ]:
#
# Representation using splines for edges
#
import polars   as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.rt_layout.parquet'

_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df  = pl.DataFrame(_lu_)
g   = rt.createNetworkXGraph(df, [('fm','to')])
pos = nx.spring_layout(g)
#rt.link(df, [('fm','to')], pos, w=800, h=600, link_shape='curve', link_size=1)

In [ ]:
df = df.with_columns(pl.col('fm').map_elements(lambda x: pos[x][0], return_dtype=pl.Float32).alias('x0'),
                     pl.col('fm').map_elements(lambda y: pos[y][1], return_dtype=pl.Float32).alias('y0'),
                     pl.col('to').map_elements(lambda x: pos[x][0], return_dtype=pl.Float32).alias('x1'),
                     pl.col('to').map_elements(lambda y: pos[y][1], return_dtype=pl.Float32).alias('y1'))
df = df.with_columns((pl.col('x1') - pl.col('x0')).alias('dx'),
                     (pl.col('y1') - pl.col('y0')).alias('dy'))
df = df.with_columns((pl.col('dx')**2 + pl.col('dy')**2).sqrt().alias('l'))
df = df.with_columns((pl.col('dx')/pl.col('l')).alias('u'),
                     (pl.col('dy')/pl.col('l')).alias('v'))
df = df.with_columns((-1.0 * pl.col('v')).alias('pu'),
                             pl.col('u') .alias('pv'))
df = df.with_columns(((pl.col('x1') - pl.col('x0')) * (1.0/3.0) + pl.col('x0') + pl.col('pu')*0.01).alias('xa'),
                     ((pl.col('y1') - pl.col('y0')) * (1.0/3.0) + pl.col('y0') + pl.col('pv')*0.01).alias('ya'),
                     ((pl.col('x1') - pl.col('x0')) * (2.0/3.0) + pl.col('x0') + pl.col('pu')*0.01).alias('xb'),
                     ((pl.col('y1') - pl.col('y0')) * (2.0/3.0) + pl.col('y0') + pl.col('pv')*0.01).alias('yb'))
# Render Operation
_ops_ = [pl.lit('<path d="M '), pl.col('x0'), pl.lit(' '), pl.col('y0'), pl.lit(' C '),
          pl.col('xa'), pl.lit(' '), pl.col('ya'), pl.lit(' '),
          pl.col('xb'), pl.lit(' '), pl.col('yb'), pl.lit(' '),
          pl.col('x1'), pl.lit(' '), pl.col('y1'), pl.lit('" fill="none" stroke="#000000" stroke-width="0.002" />')]
x0, y0, x1, y1 = min(df['x0'].min(), df['x1'].min()), min(df['y0'].min(), df['y1'].min()), max(df['x0'].max(), df['x1'].max()), max(df['y0'].max(), df['y1'].max())
svg = [f'<svg x="0" y="0" width="768" height="768" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
df = df.with_columns(pl.concat_str(_ops_).alias('svg'))
svg.extend(df['svg'].to_list())
svg.append('</svg>')
rt.tile([''.join(svg)])